In [3]:
import employeeList
from pathlib import Path
import pandas as pd
from datetime import datetime

In [4]:
dirPath = Path.home().joinpath('Desktop', '직원현황작업폴더', '202109071632')
fileName = '20210914_110909_사원명부.csv'
path = dirPath / fileName
firstDf = pd.read_csv(path, encoding = 'cp949')
dirPath2 = Path.home().joinpath('Desktop')
fileName2 = '발령일괄등록.csv'
path2 = dirPath2 / fileName2
orderDf = pd.read_csv(path2, encoding='utf-8')


firstDf = employeeList.ListPreprocessor(firstDf).operation()
lt = employeeList.ListTransformer(firstDf)
firstDic = lt.transferDict()    # get firstDic

start = datetime(2021,10,1)

orderDf = employeeList.OrderPreprocessor(orderDf).operation()
orderDf = employeeList.OrderFilter(orderDf).filter(start)
orderDf = orderDf.sort_values(by='발령일', ascending=True)
orderDic = employeeList.orderTransformer(orderDf).transferDict()     # get orderDic

firstDate = datetime(2021,1,1)

order = orderDic[0]
id = order.get('사번')
name = order.get('사원')

tracker = employeeList.Tracker(firstDate, firstDic, orderDic)

tracker.update_trackSpace()
currentDics = tracker.currentDic
currentDicsDataclassDics = tracker.create_dataclass(currentDics)

입사처리 {'사번': 202110001, '사원': '우도은', '생년월일': None, '부서': '농어촌기금운영부', '직급': '파견5급', '현직급근무일': Timestamp('2021-10-05 00:00:00'), '직책': nan, '입사일': Timestamp('2021-10-05 00:00:00'), '퇴사일': None, '사원구분': '파견직_내부', '상태': '정상근무'}
trackSpace updated : [{Timestamp('2021-10-05 00:00:00'): Dataclass(id=202110001, name='우도은', birth=None, team='농어촌기금운영부', position='파견5급', positionDate=Timestamp('2021-10-05 00:00:00'), leader=nan, joinDate=Timestamp('2021-10-05 00:00:00'), quitDate=None, kindOfContract='파견직_내부', currentState='정상근무', teamLeader='오완진', centerLeader='조태용')}]
입사처리 {'사번': 202110002, '사원': '김성웅', '생년월일': None, '부서': '경영지원부', '직급': '파견5급', '현직급근무일': Timestamp('2021-10-06 00:00:00'), '직책': nan, '입사일': Timestamp('2021-10-06 00:00:00'), '퇴사일': None, '사원구분': '파견직_내부', '상태': '정상근무'}
trackSpace updated : [{Timestamp('2021-10-06 00:00:00'): Dataclass(id=202110002, name='김성웅', birth=None, team='경영지원부', position='파견5급', positionDate=Timestamp('2021-10-06 00:00:00'), leader=nan, joinDate=Timestamp('

In [24]:
trackSpace = tracker.trackSpace
dfs = []
for key in trackSpace.keys():
    for dict in trackSpace[key]:
        dataclass = dict.values()
        df = pd.DataFrame(dataclass, index=[list(dict.keys())[0].strftime(format='%Y-%m-%d')])
        dfs.append(df)

In [27]:
trackSpaceDf = pd.concat(dfs)

cols = ['사번', '사원', '생년월일', '부서', '직급', '현직급근무일', '직책', '입사일', '퇴사일', '사원구분', '상태', '부서장(팀장)', '본부장']
trackSpaceDf.columns = cols
trackSpaceDf.to_csv(dirPath2 / 'trackSpaceDf.csv', encoding='cp949')